In [ ]:
# Packages
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
import pandas as pd
from sklearn.impute import KNNImputer
from tools import data_balancing
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import RepeatedStratifiedKFold
from skopt import BayesSearchCV
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from numpy import argmax
from sklearn.metrics import precision_recall_curve
from matplotlib import pyplot
from sklearn.inspection import permutation_importance
from sklearn.metrics import confusion_matrix
from feature_processing import feature_preprocessing, feature_processing
from tools import data_balancing, multipage, plot_permutation_importance
from sklearn.model_selection import StratifiedKFold
import pickle
import os
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import random


In [ ]:
# Set of functions 1

def __plot_correlation_matrix(correlation_matrix):
    f = plt.figure(figsize=(10, 8))
    plt.matshow(correlation_matrix, fignum=f.number)
    cb = plt.colorbar()
    cb.ax.tick_params(labelsize=14)
    # plt.title('Correlation Matrix', fontsize=16)
    # plt.tight_layout()
    return


def __missing_data_imputation(train_features, valid_features):
    """MICE = IterativeImputer(random_state=42, max_iter=1000, initial_strategy='median', tol=0.01, n_nearest_features=20)
    MICE = MICE.fit(train_features)
    train_features = MICE.transform(train_features)
    valid_features = MICE.transform(valid_features)"""
    num_neigh = 5
    print('Neighbors used for KNN imputer: ' + str(num_neigh))
    features_list_fs = train_features.columns
    KNN_imputer = KNNImputer(n_neighbors=num_neigh)
    KNN_imputer.fit(train_features)
    train_features = KNN_imputer.transform(train_features)
    valid_features = KNN_imputer.transform(valid_features)

    train_features = pd.DataFrame(data=train_features, columns=features_list_fs)
    valid_features = pd.DataFrame(data=valid_features, columns=features_list_fs)
    return train_features,  valid_features, KNN_imputer


def feature_preprocessing(train_features, valid_features):
    # a. missing values handling
    print('-----------------a. eliminate features with more than 20% of NaNs---------------')

    #   a.1. Eliminate subjects with more than a percent of missing features:

    #   a.2. Eliminate features with more than a percent of missing values:
    print('Eliminating features with more than a % of missing values...')
    feat_ini = len(train_features.columns)
    th = round((len(train_features) * 20) / 100)  # eliminate variables with more than 50% of the values missing
    train_features = train_features.dropna(thresh=th, axis='columns')
    valid_features = valid_features[train_features.columns]
    print('Features dropped : ' + str(feat_ini - len(train_features.columns)))


    # b. Outlier removal
    print('-----------------b. Outlier imputation (IQR limit and data imputation)-----------------')
    print('Outliers are being treated as NaNs after identification by IQR metrics')
    feature_list_fs = train_features.columns
    robust_scaler = RobustScaler().fit(train_features)
    train_features_rs = robust_scaler.transform(train_features)

    # find upper and lower limit to consider outliers based on the IQR
    Q1 = np.quantile(train_features_rs, 0.25, axis=0)
    Q3 = np.quantile(train_features_rs, 0.75, axis=0)
    IQR = Q3 - Q1
    lim_up = Q3 + 1.5*IQR
    lim_dow = Q1 - 1.5*IQR

    # substitute values by: (a) 0.95/0.05 Q (b) nan and then perform value imputation
    train_features[train_features_rs < lim_dow] = np.nan
    train_features[train_features_rs > lim_up] = np.nan

    train_features = pd.DataFrame(data=train_features, columns=feature_list_fs)

    print('-----------------c. Missing values imputation----------')
    #   a.3. Replace nan values with median
    print('Performing NaN imputation with KNN...')
    """medians = train_features.median()
    train_features = train_features.fillna(medians)
    valid_features = valid_features.fillna(medians)"""
    train_features, valid_features, KNN_imputer = __missing_data_imputation(train_features, valid_features)

    """print('Replacing NaN values with MICE...')
    MICE = IterativeImputer(random_state=42)
    MICE = MICE.fit(train_features)
    train_features = MICE.transform(train_features)
    valid_features = MICE.transform(valid_features)"""

    # c. scaling/standarization
    print('-----------------d. scaling/standarization----------')
    print('Applying robust scaler...')
    feature_list_fs = train_features.columns
    robust_scaler = RobustScaler().fit(train_features)
    train_features = robust_scaler.transform(train_features)
    valid_features = robust_scaler.transform(valid_features)
    train_features = pd.DataFrame(data=train_features, columns=feature_list_fs)
    valid_features = pd.DataFrame(data=valid_features, columns=feature_list_fs)

    return train_features, valid_features, robust_scaler, KNN_imputer


def __feature_selection(train_features, train_labels):
    print('-----------------d. feature selection--------------')
    print('i- Eliminating features with less than 0.01 variance. ')
    #   i.   remove constant variables
    feature_list_fs = train_features.columns
    vt = VarianceThreshold(threshold=0.01)  # eliminate constant features - Check
    temp = vt.fit_transform(train_features)
    mask = vt.get_support()
    train_features = train_features.loc[:, mask]
    print('Features dropped : ' + str(len(feature_list_fs) - len(train_features.columns)))

    #  ii. remove correlated features
    print('ii- Eliminating correlated features (>0.8). ')
    # a. visualize correlation matrix
    feature_list_fs = train_features.columns
    cor_matrix = train_features.corr().abs()
    __plot_correlation_matrix(cor_matrix)

    # b. drop correlated features
    correlated_features = set()
    correlation_matrix = train_features.corr()
    median_corr = (correlation_matrix.abs()).median(axis=0)

    for i in range(len(correlation_matrix.columns)):
        for j in range(i):
            if abs(correlation_matrix.iloc[i, j]) > 0.80:
                if median_corr[i] < median_corr[j]:
                    colname = correlation_matrix.columns[j]
                else:
                    colname = correlation_matrix.columns[i]
                correlated_features.add(colname)
    train_features.drop(labels=correlated_features, axis=1, inplace=True)
    # see how it drops correlated / drop teh one that had a lower correlation w.r to the other feat
    print('Features dropped: ' + str(len(correlated_features)))

    # c. plot correlation matrix with remaining features
    cor_matrix = train_features.corr().abs()
    feature_list_fs = list(train_features.columns)
    __plot_correlation_matrix(cor_matrix)

    # iii. recursive feature selection
    print('iii- Recurrent feature selection CV. ')
    feature_list_fs = train_features.columns
    th_select = 1
    svc = SVC(kernel="linear")
    min_features_to_select = 1
    cv_num = 3
    train_features_sb, train_labels_sb = data_balancing(train_features, train_labels)
    feats_RFE_sbs = pd.DataFrame(data=np.zeros([5, len(feature_list_fs)]), columns=feature_list_fs)
    for Nsbs in range(len(train_features_sb)):
        rfecv = RFECV(estimator=svc, step=1, cv=StratifiedKFold(cv_num), scoring='balanced_accuracy',
                      min_features_to_select=min_features_to_select, n_jobs=-1)
        rfecv.fit(train_features_sb[Nsbs], train_labels_sb[Nsbs])
        feats_RFE_sbs.iloc[Nsbs,  rfecv.support_] = int(1)
    feats_RFE_sbs = feats_RFE_sbs.sum(axis=0)
    print('Threshold for feature selection: ' + str(th_select))
    feature_list_fs = feature_list_fs[np.where(feats_RFE_sbs >= th_select)]
    train_features = train_features[feature_list_fs]
    print('Features dropped: ' + str(len(feature_list_fs) - len(train_features.columns)))

    """# iv sequencial features selection (SFFS)
    svc = SVC(kernel="linear")
    sfs = SequentialFeatureSelector(svc, n_features_to_select=None, direction='forward', scoring='balanced_accuracy',
                                    cv=StratifiedKFold(5),)
    sfs.fit(train_features, train_labels)
    train_features = train_features.iloc[:, sfs.get_support()]"""

    print('Final number of features selected : ' + str(len(train_features.columns)))

    return train_features


def feature_processing(train_features, train_labels, valid_features, feat_select_flag):
    "1. feature preprocessing"
    train_features, valid_features, robust_scaler, KNN_imputer = feature_preprocessing(train_features, valid_features)

    "2. feature selection"
    if feat_select_flag:
        print('Performing feature selection')
        train_features = __feature_selection(train_features, train_labels)
        feature_list_fs = train_features.columns
        valid_features = valid_features[feature_list_fs]

    return train_features, valid_features


def __plot_confusion_matrix(test_labels, test_predicted, labels):
    cm = confusion_matrix(test_labels, test_predicted, labels=labels)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['live', 'death'])
    disp.plot()
    plt.show()
    return


def __plot_feature_importance(clf_best, feature_list_fs, valid_features, valid_labels, plot_flag):

    "1. Impurity-based importances  (standard sklearn)"
    impu_importance = np.zeros([len(clf_best), len(feature_list_fs)])
    for Nsbs in range(len(clf_best)):
        impu_importance[Nsbs, :] = clf_best[Nsbs].feature_importances_
    impu_std = np.std(impu_importance, axis=0)
    impu_importance = np.mean(impu_importance, axis=0)

    "2. permutation importance"
    perm_importance = np.zeros([len(clf_best), len(feature_list_fs)])
    for Nsbs in range(len(clf_best)):
        perm_importance_temp = permutation_importance(clf_best[Nsbs], valid_features, valid_labels, n_jobs=-1,
                                                      n_repeats=5, random_state=42)
        perm_importance[Nsbs, :] = perm_importance_temp.importances_mean

    perm_std = np.std(perm_importance, axis=0)
    perm_importance = np.mean(perm_importance, axis=0)

    importances = np.array([impu_importance, impu_std, perm_importance, perm_std])
    feat_importance = pd.DataFrame(data=importances, columns=feature_list_fs, index=['impu_importance', 'impu_std',
                                                                                     'perm_importance', 'perm_std'])

    if plot_flag:
        "a. impurity based importances"
        sorted_idx = impu_importance.argsort()
        clf_importances = pd.Series(impu_importance[sorted_idx[::-1]], index=feature_list_fs[sorted_idx[::-1]])

        if len(feature_list_fs) >= 60:
            font_size = 6
        elif len(feature_list_fs) < 60 & len(feature_list_fs) > 40:
            font_size = 8
        elif len(feature_list_fs) <= 40:
            font_size = 10

        fig, ax = plt.subplots()
        plt.xticks(fontsize=font_size)
        plt.yticks(fontsize=font_size)
        clf_importances.plot.bar(yerr=impu_std, ax=ax)
        ax.set_title("Feature impu_importance using MDI")
        ax.set_ylabel("Mean decrease in impurity")
        fig.tight_layout()
        plt.show()

        "b. permutation importance"
        sorted_idx = perm_importance.argsort()
        fig, ax = plt.subplots()
        plt.xticks(fontsize=font_size)
        plt.yticks(fontsize=font_size)
        plt.barh(feature_list_fs[sorted_idx], perm_importance[sorted_idx], xerr=perm_std[sorted_idx])
        ax.set_title("Permutation Importance")
        fig.tight_layout()
        plt.show()

    """"3. feature importance with SHAP values"
    explainer = shap.TreeExplainer(clf_best)
    shap_values = explainer.shap_values(valid_features)
    shap.summary_plot(shap_values, valid_features, plot_type="bar")
    shap.summary_plot(shap_values, valid_features)"""

    "------------------------------------------------------"
    """# Create arrays from feature importance and feature names
    feature_importance = np.array(impu_importance[:, sorted_idx])
    feature_names = np.array(feature_list_fs)
    feature_names = feature_names[sorted_idx]
    feature_names = np.tile([feature_names], len(feature_importance)).squeeze()
    feature_importance = np.reshape(feature_importance, len(feature_names)) #

    feature_importance = feature_importance[::-1]
    feature_names = feature_names[::-1]

    # Create a DataFrame using a Dictionary
    data = {'feature_names': feature_names, 'feature_importance': feature_importance}
    fi_df = pd.DataFrame(data)

    # Sort the DataFrame in order decreasing feature importance
    # fi_df.sort_values(by=['feature_importance'], ascending=False, inplace=True)

    # Define size of bar plot
    plt.figure(figsize=(10, 8))
    plt.xticks(fontsize=font_size)
    plt.yticks(fontsize=font_size)
    # Plot Searborn bar chart
    sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'], errwidth=1)
    # Add chart labels
    plt.title('Feature importance')
    # plt.xlabel('FEATURE IMPORTANCE')
    plt.ylabel('FEATURE NAMES')
    plt.tight_layout()"""
    return feat_importance


def __model_evaluation(valid_predict, valid_labels):
    # predictions = model.predict(valid_features_df)

    tn, fp, fn, tp = confusion_matrix(valid_labels, valid_predict).ravel()

    model_metrics_1 = pd.DataFrame(
        {'acc_bal': [metrics.balanced_accuracy_score(valid_labels, valid_predict)],
         'se': [metrics.recall_score(valid_labels, valid_predict, average='binary', pos_label=1)],
         'sp': [tn / (tn + fp)],
         'PPV': [metrics.precision_score(valid_labels, valid_predict, average='binary', pos_label=1)],
         'f1_score': [metrics.f1_score(valid_labels, valid_predict, average='binary', pos_label=1)]})

    tn, fp, fn, tp = confusion_matrix((valid_labels-1) * -1, (valid_predict - 1) * -1).ravel()
    model_metrics_0 = pd.DataFrame(
        {'acc_bal': [metrics.balanced_accuracy_score(valid_labels, valid_predict)],
         'se': [metrics.recall_score(valid_labels, valid_predict, average='binary', pos_label=0)],
         'sp': [tn / (tn + fp)],
         'PPV': [metrics.precision_score(valid_labels, valid_predict, average='binary', pos_label=0)],
         'f1_score': [metrics.f1_score(valid_labels, valid_predict, average='binary', pos_label=0)]})

    print(metrics.classification_report(valid_labels, valid_predict))
    __plot_confusion_matrix(valid_labels, valid_predict, [0, 1])
    return model_metrics_1, model_metrics_0


def __model_hyperparam_tunning(train_features_sb, train_labels_sb):
    # 1. Define the values for each hyperparameter
    parameters = {"n_estimators": (10, 1000),
                  "max_depth": (1, 150),
                  "min_samples_split": (2, 10),
                  "class_weight": ['balanced_subsample']}

    # 2. Crossvalidate model to obtain the best fit
    # create classifier grid: define classifier, hyperparam values and scoring metric
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    clf = BayesSearchCV(estimator=RandomForestClassifier(random_state=42), search_spaces=parameters, cv=cv, scoring='f1_macro')
    clf.fit(train_features_sb, train_labels_sb)

    # print results for each combination of parameters
    for param, score in zip(clf.cv_results_['params'], clf.cv_results_['mean_test_score']):
        print(param, score)

    # display best combination of parameters
    print('Best combination of hyperparameters: ' + str(clf.best_params_))

    return clf


def __threshold_optimization(predict_prob, valid_labels):
    # predict probabilities
    # predict_proba = clf_best.predict_proba(valid_features)
    # keep probabilities for the positive outcome only
    # predict_proba = predict_proba[:, 1]
    # calculate roc curves
    precision, recall, thresholds = precision_recall_curve(valid_labels, predict_prob)
    # convert to f score
    fscore = (2 * precision * recall) / (precision + recall)
    fscore = np.nan_to_num(fscore)
    # locate the index of the largest f score
    ix = argmax(fscore)

    threshold_F1score = thresholds[ix]
    print('Best Threshold=%f, F-Score=%.3f' % (thresholds[ix], fscore[ix]))
    # plot the roc curve for the model
    fig, ax = plt.subplots()
    no_skill = len(valid_labels[valid_labels == 1]) / len(valid_labels)
    pyplot.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')
    pyplot.plot(recall, precision, marker='.', label='model')
    pyplot.scatter(recall[ix], precision[ix], marker='o', color='black', label='Best')
    # axis labels
    pyplot.xlabel('Recall')
    pyplot.ylabel('Precision')
    pyplot.legend()
    # show the plot
    pyplot.show()
    return threshold_F1score




def multipage(filename, figs=None, dpi=200):
    pp = PdfPages(filename)
    if figs is None:
        figs = [plt.figure(n) for n in plt.get_fignums()]
    for fig in figs:
        fig.savefig(pp, format='pdf')
    pp.close()


def data_balancing(train_features, train_labels):
    print('Total patients: ' + str(len(train_features)) + '(Class 1: ' + str(
        sum(train_labels == 1)) + ',  Class 0: ' + str(sum(train_labels == 0)), ')')
    num_sbs = 5
    random.seed(a=42, version=2)
    rand_sb = random.sample(range(0, sum(train_labels == 0)), sum(train_labels == 0))
    subset_idx_C0 = np.where(train_labels == 0)[0]
    subset_idx_C1 = np.where(train_labels == 1)[0]
    train_features_sb = list()
    train_labels_sb = list()
    step_sb = int(sum(train_labels == 0)/ num_sbs)
    ini = 0
    for Nsbs in range(num_sbs):
        fini =  ini + step_sb
        if  Nsbs==num_sbs:
            fini = sum(train_labels == 0)

        # choose the same number of 0 and 1 class to introduce into the subset
        train_features_sb.append(pd.concat([train_features.iloc[subset_idx_C0[rand_sb[ini:fini]], :],
                                       train_features.iloc[subset_idx_C1]]))

        train_labels_sb.append(np.concatenate([train_labels[subset_idx_C0[rand_sb[ini:fini]]],
                                                train_labels[subset_idx_C1]]))

        # mix 0 and 1 class randomly inside the subset
        rand_sb_2 = random.sample(range(0, len(train_features_sb[Nsbs])), len(train_features_sb[Nsbs]))
        train_features_sb[Nsbs] = train_features_sb[Nsbs].iloc[rand_sb_2, :]
        train_labels_sb[Nsbs] = train_labels_sb[Nsbs][rand_sb_2]
        ini = fini + 1

        print('S' + str(Nsbs+1) + ' - Total patients: ' + str(len(train_features_sb[Nsbs])) + '(Class 1: ' + str(
            sum(train_labels_sb[Nsbs] == 1)) + ',  Class 0: ' + str(sum(train_labels_sb[Nsbs] == 0)), ')')

    return train_features_sb, train_labels_sb


def plot_permutation_importance(model_metrics_1, per_importance, feature_list_fs):
    # Study the permutation importance for final feature selection?
    weights_median = model_metrics_1['acc_bal'] / model_metrics_1['acc_bal'].max(axis=0)
    per_importance_median = per_importance.median(axis=0)
    # per_importance_wmean = (per_importance*weights_median).sum(axis=0)/weights_median.sum()
    per_importance_std = per_importance.std(axis=0)
    sorted_idx = per_importance_median.argsort()
    fig, ax = plt.subplots()
    font_size = 6
    plt.xticks(fontsize=font_size)
    plt.yticks(fontsize=font_size)
    plt.barh(feature_list_fs[sorted_idx], per_importance_median[sorted_idx], xerr=per_importance_std[sorted_idx])
    ax.set_title("Permutation Importance")
    fig.tight_layout()
    plt.show()



In [ ]:
# Set of functions 2


def __model_training(train_features, train_labels, valid_features, valid_labels, hyper_tun_flag):
    """model = RandomForestClassifier(class_weight='balanced_subsample', random_state=42, )
    model.fit(train_features_sb, train_labels_sb)"""
    print('Model training:')

    # 1. Data balancing
    print('1. Balancing classes...')
    train_features_sb, train_labels_sb = data_balancing(train_features, train_labels)
    # create empty array that will contain the probabilities predicted with each of the models of the ensemble
    valid_predict_prob = np.zeros([len(valid_labels), 2, len(train_features_sb)])
    clf_best = list()
    for Nsbs in range(len(train_features_sb)):
        # 2. Hyperparameter tuning using bayesian optimization
        if hyper_tun_flag:
            print('E' + str(Nsbs) + ' - 2. Tuning hyperparameters using Bayesian optimization...')
            clf = __model_hyperparam_tunning(train_features_sb[Nsbs], train_labels_sb[Nsbs])
            clf_params = clf.best_params_
        else:
            print('E' + str(Nsbs) +  ' - 2. Using preset hyperparameters* ...')
            clf_params = {"n_estimators": 1000,
                          "max_depth": 101,
                          "min_samples_split": 3,
                          "class_weight": 'balanced_subsample'}

        # 3. Model training using the best parameters
        print('E' + str(Nsbs) +  ' - 3. Training the model with the best parameters...')
        # clf_best = RandomForestClassifier(**clf.best_params_)
        clf_best.append(RandomForestClassifier(**clf_params))
        clf_best[Nsbs].fit(train_features_sb[Nsbs], train_labels_sb[Nsbs])

        # 4. predict probabilities of the validation set for each model of the ensemble
        valid_predict_prob[:, :, Nsbs] = clf_best[Nsbs].predict_proba(valid_features)

    # 5. Threshold optimization
    # Average the probabilities of the different ensembles
    valid_predict_prob = np.mean(valid_predict_prob, axis=2)
    threshold_F1score = __threshold_optimization(valid_predict_prob[:, 1], valid_labels)
    valid_predict = np.zeros(len(valid_predict_prob))
    positive_out = np.where(valid_predict_prob[:, 1] > threshold_F1score)
    valid_predict[positive_out[0]] = int(1)

    # 5. Model evaluation
    print('4.Evaluating the model with the best parameters')
    feature_list_fs = train_features.columns
    valid_features = valid_features[feature_list_fs]
    model_metrics_1, model_metrics_0 = __model_evaluation(valid_predict, valid_labels)

    # 5. Plot feature importance
    plot_flag = False
    feat_importance = __plot_feature_importance(clf_best, feature_list_fs, valid_features, valid_labels, plot_flag)
    return clf_best, threshold_F1score, model_metrics_1, model_metrics_0, feat_importance


def crossvalidation_training_pipeline(kfolds_n, train_features_tv, train_labels_tv, hyper_tun_flag, feat_select_flag,
                                      perm_import_flag, saving_path):
    NKf = 0
    skf = StratifiedKFold(n_splits=kfolds_n, shuffle=True, random_state=42)  # random_state=42
    threshold_F1score = np.zeros(kfolds_n)
    feature_list = train_features_tv.columns
    feature_selection_df = pd.DataFrame(data=np.zeros([len(feature_list), kfolds_n + 1]), index=feature_list)
    model_metrics_1 = pd.DataFrame(data=np.zeros([kfolds_n, 5]), columns=['acc_bal', 'se',
                                                                          'sp', 'PPV', 'f1_score'])

    model_metrics_0 = pd.DataFrame(data=np.zeros([kfolds_n, 5]), columns=['acc_bal', 'se',
                                                                          'sp', 'PPV', 'f1_score'])

    per_importance = pd.DataFrame(data=np.zeros([kfolds_n, len(feature_list)]), columns=feature_list)
    for train_index, valid_index in skf.split(train_features_tv, train_labels_tv):
        # print("TRAIN:", train_index, "TEST:", valid_index)
        train_features, valid_features = train_features_tv.iloc[train_index, :], train_features_tv.iloc[valid_index, :]
        train_labels, valid_labels = train_labels_tv[train_index], train_labels_tv[valid_index]

        # Inside cross-validation
        print('Training Features Shape:', train_features.shape, 'Training Labels Shape:', train_labels.shape)
        print('Validation Features Shape:', valid_features.shape, 'Validation Labels Shape:', valid_labels.shape)

        "b. feature selection"
        train_features, valid_features = feature_processing(train_features, train_labels, valid_features,
                                                                  feat_select_flag)
        feature_list_fs = list(train_features.columns)
        feature_selection_df.loc[feature_list_fs, NKf] = 1

        "c. Model training"
        model, threshold_F1score_temp, model_metrics_1_temp, model_metrics_0_temp, feat_importance = \
            __model_training(train_features, train_labels, valid_features, valid_labels, hyper_tun_flag)

        model_metrics_1.iloc[NKf, :] = model_metrics_1_temp
        model_metrics_0.iloc[NKf, :] = model_metrics_0_temp
        threshold_F1score[NKf] = threshold_F1score_temp
        if perm_import_flag:
            per_importance.iloc[NKf, :] = feat_importance.loc['perm_importance']


        "d. Saving parameters"
        saving_path_temp = os.path.join(saving_path, 'KFold_' + str(NKf + 1))
        if not os.path.exists(saving_path_temp):
            os.makedirs(saving_path_temp)

        # 1. save features selected
        pickle.dump(feature_list_fs, open(os.path.join(saving_path_temp, 'features_selected.pkl'), "wb"))
        # 2. save trained model
        pickle.dump(model, open(os.path.join(saving_path_temp, 'random_forest.sav'), "wb"))
        # 3. save threshold
        pickle.dump(threshold_F1score_temp, open(os.path.join(saving_path_temp, 'threshold_F1score.pkl'), "wb"))
        # 4. save figures
        multipage(os.path.join(saving_path_temp, 'figures_K' + str(NKf + 1) + '.pdf'))
        plt.close('all')
        # 5. current model metrics
        pickle.dump(model_metrics_1_temp, open(os.path.join(saving_path_temp, 'model_metrics_1.pkl'), "wb"))
        # 6. save current train-validation split
        pickle.dump(train_index, open(os.path.join(saving_path_temp, 'train_index.pkl'), "wb"))
        pickle.dump(valid_index, open(os.path.join(saving_path_temp, 'valid_index.pkl'), "wb"))
        # 7. Save features importance
        feat_importance.to_csv((os.path.join(saving_path_temp, 'feature_importance.csv')))
        NKf = NKf + 1

    features_Kfold = feature_selection_df.sum(axis=1)
    features_Kfold.hist()

    "e. median permutation importance"
    if perm_import_flag:
        plot_permutation_importance(model_metrics_1, per_importance, feature_list_fs)

    "f. save output of full k-fold"
    saving_path_temp = os.path.join(saving_path, 'all')
    if not os.path.exists(saving_path_temp):
        os.makedirs(saving_path_temp)

    model_metrics_kfold = pd.DataFrame({'class_0_mean': model_metrics_0.mean(), 'class_0_std': model_metrics_0.std(),
                                        'class_1_mean': model_metrics_1.mean(), 'class_1_std': model_metrics_1.std()})

    model_metrics_1.to_csv((os.path.join(saving_path_temp, 'model_metrics_1.csv')))
    model_metrics_0.to_csv((os.path.join(saving_path_temp, 'model_metrics_0.csv')))
    model_metrics_kfold.to_csv((os.path.join(saving_path_temp, 'model_metrics_kfold.csv')))
    feature_selection_df.to_csv((os.path.join(saving_path_temp, 'features_selected.csv')))
    pickle.dump(threshold_F1score, open(os.path.join(saving_path_temp, 'threshold_F1score.pkl'), "wb"))

    multipage(os.path.join(saving_path_temp, 'figures.pdf'))
    plt.close('all')

    return feature_selection_df, threshold_F1score, model_metrics_kfold, model_metrics_1, model_metrics_0, per_importance


def model_testing(train_features_tv, train_labels_tv, test_features, test_labels, feature_list_fs, threshold_final,
                  saving_path):
    # feature_list = train_features_tv.columns
    # -----------------I. Data preparation for model testing----------------------
    # 1. Feature selection
    "Chosen features"
    # feature_list_fs = train_features_sb.columns

    "select only chosen features"
    test_features = test_features[feature_list_fs]
    train_features_tv = train_features_tv[feature_list_fs]

    # 2. feature processing
    train_features_tv, test_features, robust_scaler, KNN_imputer = feature_preprocessing(train_features_tv,
                                                                                         test_features)

    # 3. Training data balancing
    print('1. Balancing classes...')
    train_features_sb, train_labels_sb = data_balancing(train_features_tv, train_labels_tv)
    # create empty array that will contain the probabilities predicted with each of the models of the ensemble
    train_predict_prob = np.zeros([len(train_labels_tv), 2, len(train_features_sb)])
    test_predict_prob = np.zeros([len(test_labels), 2, len(train_features_sb)])

    clf_final = list()
    for Nsbs in range(len(train_features_sb)):
        # 2. Hyperparameter tuning using bayesian optimization
        print('E' + str(Nsbs) + ' - 2. Using preset hyperparameters* ...')
        clf_params = {"n_estimators": 1000,
                      "max_depth": 101,
                      "min_samples_split": 3,
                      "class_weight": 'balanced_subsample'}

        # 3. Model training using the best parameters
        print('E' + str(Nsbs) + ' - 3. Training the model with the best parameters...')
        # clf_final = RandomForestClassifier(**clf.best_params_)
        clf_final.append(RandomForestClassifier(**clf_params))
        clf_final[Nsbs].fit(train_features_sb[Nsbs], train_labels_sb[Nsbs])

        # 4. predict probabilities of the validation set for each model of the ensemble
        train_predict_prob[:, :, Nsbs] = clf_final[Nsbs].predict_proba(train_features_tv)
        test_predict_prob[:, :, Nsbs] = clf_final[Nsbs].predict_proba(test_features)

        print('Total patients: ' + str(len(train_features_sb[Nsbs])) + '(Class 1: ' + str(
            sum(train_labels_sb[Nsbs] == 1)) +
              ',  Class 0: ' + str(sum(train_labels_sb[Nsbs] == 0)), ')')

    # -----------------III. Model testing----------------------
    # test model
    # training score
    print('-------------Train---------------')
    train_predict_prob = np.mean(train_predict_prob, axis=2)
    train_predict = np.zeros(len(train_predict_prob))
    positive_out = np.where(train_predict_prob[:, 1] > threshold_final)
    train_predict[positive_out[0]] = int(1)
    train_metrics_1, train_metrics_0 = __model_evaluation(train_predict, train_labels_tv)

    # testing score
    print('-------------Test---------------')
    test_predict_prob = np.mean(test_predict_prob, axis=2)
    test_predict = np.zeros(len(test_predict_prob))
    positive_out = np.where(test_predict_prob[:, 1] > threshold_final)
    test_predict[positive_out[0]] = int(1)
    test_metrics_1, test_metrics_0 = __model_evaluation(test_predict, test_labels)

    metrics_model = pd.concat([train_metrics_0, train_metrics_1, test_metrics_0, test_metrics_1])
    metrics_model.index = ["train_0", "train_1", "test_0", "test_1"]

    "f. Saving test parameters"
    if not os.path.exists(saving_path):
        os.makedirs(saving_path)

    # 1. save features selected
    pickle.dump(feature_list_fs, open(os.path.join(saving_path, 'features_selected.pkl'), "wb"))
    # 2. save trained model
    pickle.dump(clf_final, open(os.path.join(saving_path, 'random_forest_ensemble.sav'), "wb"))
    # 3. save threshold
    pickle.dump(threshold_final, open(os.path.join(saving_path, 'threshold_F1score.pkl'), "wb"))
    # 4. save figures
    multipage(os.path.join(saving_path, 'figures_final.pdf'))
    # 5. model metrics
    pickle.dump(metrics_model, open(os.path.join(saving_path, 'metrics_model.pkl'), "wb"))
    # 6. robust scaler
    pickle.dump(robust_scaler, open(os.path.join(saving_path, 'robust_scaler.pkl'), "wb"))
    # 7. KNN imputer
    pickle.dump(KNN_imputer, open(os.path.join(saving_path, 'KNN_imputer.pkl'), "wb"))

    return clf_final, train_metrics_1, train_metrics_0, test_metrics_1, test_metrics_0


In [ ]:

# from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os

from parsel_files import parsel_files_f
from model_train_test import model_testing, crossvalidation_training_pipeline


saving_path = 'G:\\Mi unidad\\3_POLIMI\\A_Projects\\11P_COVIDSQUARE\\Analysis\\Feature_analysis\\output\\'
" I. load data from csv file"
features = pd.read_csv('data.csv')

labels = np.array(features['Label'])
features = features.set_index('Record_Name')
"II. Model ------------------------------------------------------------------------------------------------------------"

"a. Train - Test split"
# elim = [33, 99, 272, 367, 437, 489, 671, 747, 772, 854, 1029, 1121] no data on glasgow and bravo reports
# Read test IDs.
test_IDs = list(pd.read_csv('validation_records.csv').squeeze().sort_values())
# Select features and labels from the corresponding test IDs
test_features = features.loc[test_IDs]
test_labels = np.array(test_features['Label'])
test_features = test_features.drop(columns=['Label'])
print('Testing Features Shape:', test_features.shape, 'Testing Labels Shape:', test_labels.shape)

# Eliminate test patients from the training and validation dataset 
train_features_tv = features.drop(features.loc[test_IDs].index)
train_labels_tv = np.array(train_features_tv['Label'])
train_features_tv = train_features_tv.drop(columns=['Label'])


"b. Crossvalidating for feature selection and cv scores"
# use only training: split train-validation
run_folder = 'P4'
cv_type = ['RFE', 'PI', 'final']
hyper_tun_flag = [False, False, False]
feat_select_flag = [True, False, False]
perm_import_flag = [False, True, True]

Threshold_RFE = 3
Threshold_PI = 0
kfolds_n = 10

# still there is an unsolved error in the crossvalidation function
for cv_t in range(len(cv_type)):
    print('----------------CV:  ' + cv_type[cv_t] + '----------------')
    "b. cross-validation training with feature selection"
    saving_path_temp = os.path.join(saving_path, run_folder, 'CV', cv_type[cv_t])

    feature_selection_df, threshold_F1score, model_metrics_kfold, model_metrics_1, model_metrics_0, per_importance = \
        crossvalidation_training_pipeline(kfolds_n, train_features_tv, train_labels_tv, hyper_tun_flag[cv_t],
                                          feat_select_flag[cv_t], perm_import_flag[cv_t],  saving_path_temp)

    if cv_t == 0:
        # Decide set of features
        features_Kfold = feature_selection_df.sum(axis=1)
        features_selection = features_Kfold.iloc[np.where(features_Kfold >= Threshold_RFE)]
        feature_list_fs = features_selection.index
        train_features_tv = train_features_tv[feature_list_fs]

    elif cv_t == 1:
        # Decide set of features
        per_importance_median = per_importance.median(axis=0)
        features_selection = per_importance_median[per_importance_median > Threshold_PI]
        feature_list_fs = features_selection.index
        train_features_tv = train_features_tv[feature_list_fs]


"d. Model testing"
# Decide final threshold
threshold_final = np.median(threshold_F1score)
print('Testing model using Threshold for number of features in Kfold: ' + str(Threshold_RFE) +
      'and Threshold f1score: ' + str(threshold_final))

saving_path = os.path.join(saving_path, run_folder, 'test')
clf_final, train_metrics_1, train_metrics_0, test_metrics_1, test_metrics_0 = \
    model_testing(train_features_tv, train_labels_tv, test_features, test_labels, feature_list_fs, threshold_final,
                  saving_path)
